![example](images/director_shot.jpeg)

# Microsoft Movie Profitability Analysis by Genre ___ etc etc

**Author:** Spencer Hadel
***

## Overview

This analysis will review recent movie data, and deem which decisions yield the most profitable results. Microsoft can use this data to embark on it's own journey of video content production, while informed of what practices to adhere to in an attempt to create the best possible content that will gain the best possible return on investment as well as gain positive customer reviews.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [515]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

budgets: for release date, production budget, and domestic/worldwide gross

titles: movie names and genres

names: for actors, directors, etc (connect to title_basics via known_for_titles(?))

reviews: ratings, also connected through tconst

In [516]:
budgets_df = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz')
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


use release_date to make new columns MONTH and YEAR
#probably also remove years before a certain point.

In [517]:
budgets_df['month'] = budgets_df.release_date.str[:3]
budgets_df['year'] = budgets_df.release_date.str[-4:]
budgets_df.drop(['id','domestic_gross', 'release_date'], axis=1, inplace=True)

In [518]:
months_to_nums = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05',
            'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

budgets_df['month_num'] = budgets_df['month'].apply(lambda x: months_to_nums.get(x))

change month column to ints

In [519]:
budgets_df.head()

,movie,production_budget,worldwide_gross,month,year,month_num
0,Avatar,"$425,000,000","$2,776,345,279",Dec,2009,12
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$1,045,663,875",May,2011,05
2,Dark Phoenix,"$350,000,000","$149,762,350",Jun,2019,06
3,Avengers: Age of Ultron,"$330,600,000","$1,403,013,963",May,2015,05
4,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$1,316,721,747",Dec,2017,12


In [520]:
budgets_df.rename(columns={'production_budget': 'budget', 'worldwide_gross': 'gross'}, inplace=True)

Change budget, domestic gross, and year to INTs

In [521]:
budgets_df['budget'].replace('[\$,]', '', regex=True, inplace=True)
budgets_df['gross'].replace('[\$,]', '', regex=True, inplace=True)

budgets_df['year'] = pd.to_numeric(budgets_df['year'])
budgets_df['budget'] = pd.to_numeric(budgets_df['budget'])
budgets_df['gross'] = pd.to_numeric(budgets_df['gross'])

In [522]:
budgets_df.dtypes

movie        object
budget        int64
gross         int64
month        object
year          int64
month_num    object
dtype: object

Add a profit column using budget and gross

In [523]:
budgets_df['profit'] = budgets_df["gross"] - budgets_df["budget"]

In [524]:
titles_df = pd.read_csv('data/zippedData/imdb.title.basics.csv.gz')
titles_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


#remove start_year and runtime, as well as worldwide gross (we are looking at DOMESTIC numbers foe this analysis)


In [525]:
titles_df.drop(['start_year', 'runtime_minutes', 'original_title'], axis=1, inplace=True)

In [526]:
combined_df = pd.merge(titles_df, budgets_df, left_on='primary_title', right_on='movie', how = 'inner')
combined_df.drop(['primary_title'], axis=1, inplace = True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3815 entries, 0 to 3814
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tconst     3815 non-null   object
 1   genres     3743 non-null   object
 2   movie      3815 non-null   object
 3   budget     3815 non-null   int64 
 4   gross      3815 non-null   int64 
 5   month      3815 non-null   object
 6   year       3815 non-null   int64 
 7   month_num  3815 non-null   object
 8   profit     3815 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 298.0+ KB


Drop rows with incomplete info, or where movie is a duplicate.

Also dropping rows with $0 listed as domestic gross income, because this is clearly incomplete data that could drastically skew results.

In [527]:
combined_df.dropna(inplace=True)
combined_df.drop_duplicates('movie')
combined_df= combined_df[combined_df['gross'] != 0]
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3379 entries, 0 to 3813
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tconst     3379 non-null   object
 1   genres     3379 non-null   object
 2   movie      3379 non-null   object
 3   budget     3379 non-null   int64 
 4   gross      3379 non-null   int64 
 5   month      3379 non-null   object
 6   year       3379 non-null   int64 
 7   month_num  3379 non-null   object
 8   profit     3379 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 264.0+ KB


Finally we will sort the columns for better readability

In [528]:
combined_df = combined_df[['tconst', 'movie', 'genres', 'budget', 'gross', 'profit', 'month', 'year', 'month_num']]

Let's only look at the last 4 years of movies. First we look at the dataframe starting with the most recent movie for better understanding.

In [529]:
combined_df.sort_values(by='year').tail()

,tconst,movie,genres,budget,gross,profit,month,year,month_num
2899,tt6857112,Us,"Horror,Mystery,Thriller",20000000,254210310,234210310,Mar,2019,03
2014,tt6476140,Serenity,"Drama,Mystery,Sci-Fi",25000000,11367029,-13632971,Jan,2019,01
1745,tt1773741,The Intruder,Horror,8000000,36005871,28005871,May,2019,05
1746,tt6722030,The Intruder,"Drama,Horror,Mystery",8000000,36005871,28005871,May,2019,05
2707,tt2392373,Kursk,Drama,40000000,4212799,-35787201,Jun,2019,06


So we will look at all movies in ther years 2016-2019.

For now, we'll look at each year individually.

In [530]:
df_2016 = combined_df[combined_df['year'] == 2016].sort_values(by='month_num', ascending = True)
df_2017 = combined_df[combined_df['year'] == 2017].sort_values(by='month_num', ascending = True)
df_2018 = combined_df[combined_df['year'] == 2018].sort_values(by='month_num', ascending = True)
df_2019 = combined_df[combined_df['year'] == 2019].sort_values(by='month_num', ascending = True)

make sure order of months comes out correctly in graphs

FIRST, We make charts of each year's gross, and profit, by month

plotting grrrr

create graph showing each years total profits, followed by a group of graphs for the last 4-5 years by month.

identify most common genres of film with value_counts

replace genre names with more understandable names

create graph showing genre correlation to profitability

        by year, then by month in a year
        
        then show by month in other years to identify trend

identify which genres are most profitable at which times of year

demonstrate what the most profitable time of year is in general, but make sure to indicate genre, time of year

also identify production budget, especially for highest or lowest times of year
        this will make clear what kinds of projects to commit to in slower seasons, etc.

after all this, using our data about profitability, pick the films with highest ratings to help inform decisions further.

add reviews column

In [6]:
reviews_df = pd.read_csv('data/zippedData/imdb.title.ratings.csv.gz')
reviews_df.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


combine the following table with existing data in a new df

In [40]:
# Here you run your code to explore the data

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [41]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [42]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***